# Collaborative Filtering with Matrix Factorization

[Collaborative filtering](https://en.wikipedia.org/wiki/Collaborative_filtering) (CF) aims to fill in the missing entries of a user-item rating matrix with predicted ratings, so that users are recomended new items based on the predicted ratings. 

In generally, there are two approaches for collaborative filtering: memory-based CF and model-based CF. In this notebook, we will focus on model-based approach, in which models are developed using different machine learning algorithms to predict users' rating of unrated items. Here we will introduce the most popular matrix factorization model.


<img src="https://github.com/yin-penghang/AMAT593/blob/main/figs/15_CF.png?raw=true" width = '400'>

## Matrix Factorization

Model-based collaborative filtering is an application of matrix factorization to identify the relationship between items’ and users’ entities. With the input of users’ ratings on the items, we would like to predict how the users would rate the items so the users can get the recommendation based on the prediction.

Suppose we have the user-item rating matrix $\mathbf{R}$ for $m$ users and $n$ items, and the ratings are integers ranging from 1 to 5:

<img src= 'https://github.com/yin-penghang/AMAT593/blob/main/figs/15_rating_matrix.png?raw=true' width = '800'>

The $(i,j)$-th entry $r_{i,j}$ represents the $i$-th user's rating on the $j$-th item.
The rating matrix $\mathbf{R}\in\mathbb{R}^{m\times n}$ has a large portion of entries/ratings missing, and our goal here is to impute the full matrix. 

Matrix factorization model assumes: every item has the same set of $k$ **latent factors** based on which a user will give the rating, where **$k$ is much less than the number of users and the number of items**. Maybe one factor means "movies with frantic chases", another factor might mean "movies with a plot twist", etc. 

Let $\mathbf{q}_j \in\mathbb{R}^k$ be the $j$-th item's **components** in the $k$ latent factors and $\mathbf{p}_i\in\mathbb{R}^k$ be the $i$-th user's **personal preferences** on these $k$ latent factors, e.g.,

    Movie j = 0.5 x frantic chases + 0.2 x plot twist + ...
    
    User i = 0.3 x fan of frantic chases + 1.8 x fan of plot twist + ...

In practice, the latent factors are opaque. Given $\mathbf{q}_j$ and $\mathbf{p}_i$, the model generates the rating:

$$
\hat{r}_{i,j} = \mathbf{p}_i^\top \mathbf{q}_j
$$

In matrix form, denote by $\mathbf{P} = \begin{bmatrix} -  \mathbf{p}_1^\top - \\ \cdots \\ -  \mathbf{p}_m^\top - \end{bmatrix}\in\mathbb{R}^{m\times k}$ the user latent matrix (with transpose), and $\mathbf{Q} = \begin{bmatrix} | \qquad | \\ \mathbf{q}_1 \cdots \mathbf{q}_n \\ | \qquad | \end{bmatrix}\in\mathbb{R}^{k\times n}$ the item latent matrix, then the predicted rating matrix has the **low-rank matrix factorization**

$$
\hat{\mathbf{R}} = \mathbf{P} \, \mathbf{Q}\in\mathbb{R}^{m\times n}
$$

<img src='https://github.com/yin-penghang/AMAT593/blob/main/figs/15_MatFacto.png?raw=true' width = '500'>

For any observed rating $r_{i,j}\in\mathbf{R}$, $\hat{\mathbf{R}}$ satisfies 

$$
\hat{r}_{i,j} = \mathbf{p}_i^\top \mathbf{q}_j = r_{i,j}.
$$

**Under the constraints above, we aim to infer the user latent matrix $\mathbf{P}$ and item latent matrix $\mathbf{Q}$** by solving

$$
\min_{\{\mathbf{p}_i\}_{i=1}^m, \{\mathbf{q}_j\}_{j=1}^n} \; \sum_{\mbox{ observed } r_{i,j}} (r_{i,j} - \mathbf{p}_i^\top \mathbf{q}_j)^2
+ \lambda(\|\mathbf{p}_i\|^2 + \|\mathbf{q}_j\|^2)
$$

The model could be more sophisticated by factoring in, for example, the bias of each user towards the rating system. 

### The Matrix Factorization Algorithm

The minimization is performed by a straightforward stochastic gradient descent: for observed $r_{i,j}$, iterate

\begin{align}
\mathbf{p}_i \leftarrow \mathbf{p}_i - \eta  \left( (\mathbf{p}_i^\top \mathbf{q}_j - r_{i,j})\mathbf{q}_j + \lambda \mathbf{p}_i \right) \\
\mathbf{q}_j \leftarrow \mathbf{q}_j - \eta  \left( (\mathbf{p}_i^\top \mathbf{q}_j - r_{i,j})\mathbf{p}_i + \lambda \mathbf{q}_j \right)
\end{align}

The above iterations are known as the SVD algorithm, as popularized by [Simon Funk](https://sifter.org/~simon/journal/20061211.html) during the Netflix Prize, although 
**it is actually NOT [SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition) which computes the factorization of a full matrix.** The more appropriate term for what Funk did is **matrix factorization**.

## Algorithm Implementation

Our matrix factorization implementation is based on [``surprise``](https://surprise.readthedocs.io/en/stable/index.html#) library.

### Surprise Library

``surprise`` is a Python scikit for building and analyzing recommender systems that deal with explicit rating data.

* give users perfect control over their experiments.

* alleviate the pain of Dataset handling. Users can use both built-in datasets (Movielens, Jester), and custom datasets.

* provide various ready-to-use prediction algorithms and various built-in similarity measures.

* make it easy to implement new algorithm ideas.

* similar to ``sklearn``, provide tools to evaluate, analyse and compare the algorithms’ performance, such as cross-validation and hyperparameter search.

In [31]:
import pandas as pd
import numpy as np

import surprise
from surprise import Dataset
from surprise import Reader
import warnings; warnings.simplefilter('ignore')

The custum algorithm is a class derived from ``AlgoBase`` that has an ``estimate`` method. ``AlgoBase`` is an abstract class that defines the basic behavior of a prediciton algorithm.

In [3]:
class MatrixFacto(surprise.AlgoBase):
    '''A basic rating prediction algorithm based on matrix factorization.'''
    
    def __init__(self, n_factors, n_epochs,  learning_rate=0.05, reg_param=0.0):
        
        self.lr = learning_rate  # learning rate for SGD
        self.n_epochs = n_epochs  # number of iterations of SGD
        self.n_factors = n_factors  # number of factors
        self.lam = reg_param # regularization parameter
        
    def fit(self, trainset):
        
        print('Fitting data with SGD...')
        
        # Randomly initialize the user and item factors.
        p = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        q = np.random.normal(0, .1, (trainset.n_items, self.n_factors))
        
        # SGD procedure
        for _ in range(self.n_epochs):
            for i, j, r_ij in trainset.all_ratings():
                err =  np.dot(p[i], q[j]) - r_ij
                # Update vectors p_u and q_i
                p[i] -= self.lr * (err * q[j] + self.lam * p[i])
                q[j] -= self.lr * (err * p[i] + self.lam * q[j])
        
        self.p, self.q = p, q
        self.trainset = trainset

    def estimate(self, i, j):
        '''Return the estmimated rating of user u for item i.'''
        
        # return scalar product between p_i and q_j if user and item are known,
        # return np.dot(self.p[i], self.q[j])
        
        if self.trainset.knows_user(i) and self.trainset.knows_item(j):
            return np.dot(self.p[i], self.q[j])
        else:
            return self.trainset.global_mean

## MovieLens Dataset

MovieLens small dataset contains ~100,000 ratings from ~600 users on ~9700 movies.

In [32]:
# Reading ratings file
ratings = pd.read_csv('ratings.csv', sep=',', encoding='latin-1', usecols=['userId','movieId','rating'])

# Reading movies file
movies = pd.read_csv('movies.csv', sep=',', encoding='latin-1', usecols=['movieId','title','genres'])

In [33]:
ratings.head(10)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
5,1,70,3.0
6,1,101,5.0
7,1,110,4.0
8,1,151,5.0
9,1,157,5.0


In [34]:
n_users = ratings.userId.unique().shape[0]
n_movies = ratings.movieId.unique().shape[0]
print('Number of users = ' + str(n_users) + '\nNumber of movies = ' + str(n_movies))

Number of users = 610
Number of movies = 9724


We load the data from ``DataFrame`` using ``surprie``'s ``Dataset.load_from_df`` function:

In [35]:
# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

Let's first check the sparsity of the ratings dataset, i.e., the percentage of missing ratings:

In [36]:
sparsity = round(1.0 - len(ratings) / float(n_users * n_movies), 3)
print('The sparsity level of MovieLens dataset is ' +  str(sparsity * 100) + '%')

The sparsity level of MovieLens dataset is 98.3%


Train test split for the dataset, use 80% of the ratings for training:

In [37]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=.2, random_state=101)

Instantiate a ``MatrixFacto``  algorithm object with specified hyperparameters:

In [38]:
algo_MF = MatrixFacto(n_factors=50, n_epochs=30, learning_rate=.02, reg_param = 0.1)

### Model evaluation

In [39]:
from surprise import accuracy
algo_MF.fit(trainset)

Fitting data with SGD...


In [40]:
test_pred = algo_MF.test(testset)
print("Matrix factorization test RMSE: {0:0.4f}".format(accuracy.rmse(test_pred, verbose=False)))

Matrix factorization test RMSE: 0.8939


### Prediction

In [41]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
227,1,3744,4.0
228,1,3793,5.0
229,1,3809,4.0
230,1,4006,4.0


Now let's predict the rating that User with ID 1 will give to a random movie, say with Movie ID 2.

In [42]:
algo_MF.estimate(1, 2)

2.8281169219871045

For movie with ID 1994, the estimated prediction of 3.76. The recommender system works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have rated the movie.

In [43]:
algo_MF.estimate(1, 1994)

3.572940766821093

In [44]:
algo_MF.estimate(1, 1) # true rating 4

4.0845820861059625

In [45]:
algo_MF.estimate(1, 3809) # true rating 4

2.3438209325493737

In [46]:
algo_MF.estimate(1, 47) # true rating 5

3.147145998924543

In [47]:
algo_MF.estimate(1, 4006) # true rating 4

3.1981566007142366

## ``surprise`` SVD Built-in Method

``surprise``'s ``SVD`` implementation is more sophisticated than ours, which takes into account the so-called user bias and item bias. See more details [here](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp).

In [48]:
from surprise import SVD

algo_svd = SVD(n_factors=50, n_epochs=30, lr_all=0.02, reg_all=0.1)

algo_svd.fit(trainset)
test_pred = algo_svd.test(testset)
print("Surprise's SVD Test RMSE: : {0:0.4f}".format(accuracy.rmse(test_pred, verbose=False)))

Surprise's SVD Test RMSE: : 0.8505


In [49]:
algo_svd.estimate(1, 2)

2.8882236967411625

In [50]:
algo_svd.estimate(1, 1) # true rating 4

3.50999235117119

In [51]:
algo_svd.estimate(1, 47) # true rating 5

3.277442693345131

In [52]:
algo_svd.estimate(1, 3809) # true rating 4

2.529485117823854

In [53]:
algo_svd.estimate(1, 4006) # true rating 4

3.394018813129909